# **Typologies DF**

## Expected Features

| features name | type  | description  |
|----	|---	|---
| `building` | str | unique id for specific building
| `asset` | str | describes the programme of the building
| `typology` | str | describes the sub category of that building asset
| `plot` | str | specific plot code for building
| `area` | float | total floor area of building
| `gfa` | float | area calculated by applyin efficiency to area 
| `efficiency` | float | float value representing the efficiency as a percentage of 100 (1.0 = 100%, 0.5 = 50%, 0.1 = 10% )
| `facade` | float | value representing the envelope area
| `height` | float | value representing the height of building
| `floors` | int | value representing the number of floors in a building

the dataframe is amplified by the specailist dataset: `brief_metrics`\
Typology dataframes shoud contain the building typologies as well as any other location based identification such as plot zone and building types. the naming should be informed by the specialist dataset 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# import modules
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plot

In [3]:
from etl.extract import ProjectZero

In [4]:
# import projectzero data
from etl.extract import ProjectZero
data = ProjectZero().get_data()

# view keys
data.keys()

# df_model instance
df_model = data['hz_model'].copy()
df_model.head()

c:\Users\hramzan\Documents\GitHub\project_zero\etl\extract.py:24: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  for (key, df) in zip(key_names, [pd.read_csv(os.path.join(csv_path, file)) for file in file_names]):


,ID,Typology,Area,Plot,Building,Colour
0,be49f53d-2a79-4d59-a0f4-39cfbccac1ef,Commercial,1696.329992,YH090703-05,,#FF8DA2
1,186a8323-9d4e-4307-8d0a-32207f5acf58,Commercial,2896.125039,YH090703-04,Tower 03,#FF8DA2
2,2fce15ca-9547-40c0-b47c-831838e39a1f,Commercial,2056.249041,YH090703-04,Tower 03,#FF8DA2
3,179dbd06-4328-4b56-8b6c-e3bdf2979253,Commercial,2056.249041,YH090703-04,Tower 03,#FF8DA2
4,e8dfed03-a169-40ae-a274-c37d146433d9,Commercial,1696.329992,YH090703-05,,#FF8DA2


## 1. `get_building_typology`

this method should return a dataframe that has the following features:\
`building`, `asset`, `typology`(optional), `plot`, `Area`

In [5]:
# Drop unecessary columns
df_model = df_model.drop(columns=['ID','Colour'])

In [6]:
df_model.head()

,Typology,Area,Plot,Building
0,Commercial,1696.329992,YH090703-05,
1,Commercial,2896.125039,YH090703-04,Tower 03
2,Commercial,2056.249041,YH090703-04,Tower 03
3,Commercial,2056.249041,YH090703-04,Tower 03
4,Commercial,1696.329992,YH090703-05,


In [7]:
# filtering str based columns 
columns = list(df_model.columns)
str_columns = [column for column in columns if columns != 'Area']

# replace empty with unassigned
for column in str_columns:
    df_model[column] = df_model[column].replace(' ','')


In [8]:
# dropping empty values for building column
df_model = df_model[df_model.Building != '']

df_model

,Typology,Area,Plot,Building
1,Commercial,2896.125039,YH090703-04,Tower 03
2,Commercial,2056.249041,YH090703-04,Tower 03
3,Commercial,2056.249041,YH090703-04,Tower 03
5,Commercial,2896.125039,YH090703-04,Tower 03
6,Commercial,2896.125039,YH090703-04,Tower 03
...,...,...,...,...
564,Retail,70.068490,YH090703-01,Podium
565,Retail,106.068490,YH090703-01,Podium
566,Retail,70.068583,YH090703-01,Podium
567,Retail,192.468583,YH090703-01,Podium


In [9]:
# Group by building type 
df_grouped = df_model.groupby(['Building', 'Typology','Plot']).sum(False).reset_index().sort_values('Building')
df_grouped.Area = df_grouped.Area.astype(int)
df_grouped

,Building,Typology,Plot,Area
0,Podium,Commercial,YH090703-07,9938
1,Podium,Landscape,YH090703-01,3194
2,Podium,Landscape,YH090703-04,5805
3,Podium,Landscape,YH090703-07,4288
4,Podium,Plaza,YH090703-01,8870
5,Podium,Plaza,YH090703-04,10758
6,Podium,Plaza,YH090703-07,14475
7,Podium,Retail,YH090703-01,12776
8,Podium,Retail,YH090703-04,24729
9,Podium,Retail,YH090703-07,13042


## 2. `get_building_facade`

this method should return a dataframe that has the following features: \
`building`, `area`, `facade`

In [10]:
# df_model instance
df_model = data['hz_model'].copy()
df_model.head()

,ID,Typology,Area,Plot,Building,Colour
0,be49f53d-2a79-4d59-a0f4-39cfbccac1ef,Commercial,1696.329992,YH090703-05,,#FF8DA2
1,186a8323-9d4e-4307-8d0a-32207f5acf58,Commercial,2896.125039,YH090703-04,Tower 03,#FF8DA2
2,2fce15ca-9547-40c0-b47c-831838e39a1f,Commercial,2056.249041,YH090703-04,Tower 03,#FF8DA2
3,179dbd06-4328-4b56-8b6c-e3bdf2979253,Commercial,2056.249041,YH090703-04,Tower 03,#FF8DA2
4,e8dfed03-a169-40ae-a274-c37d146433d9,Commercial,1696.329992,YH090703-05,,#FF8DA2


In [11]:
from etl.utils import estimate_facade_area

# if facade area not in df then generate estimate with utils.estimate_facade_area
if 'Facade' not in list(df_model.columns):
    df_model['Facade'] = estimate_facade_area(df_model, 4)

df_model.head()

,ID,Typology,Area,Plot,Building,Colour,Facade
0,be49f53d-2a79-4d59-a0f4-39cfbccac1ef,Commercial,1696.329992,YH090703-05,,#FF8DA2,658
1,186a8323-9d4e-4307-8d0a-32207f5acf58,Commercial,2896.125039,YH090703-04,Tower 03,#FF8DA2,861
2,2fce15ca-9547-40c0-b47c-831838e39a1f,Commercial,2056.249041,YH090703-04,Tower 03,#FF8DA2,725
3,179dbd06-4328-4b56-8b6c-e3bdf2979253,Commercial,2056.249041,YH090703-04,Tower 03,#FF8DA2,725
4,e8dfed03-a169-40ae-a274-c37d146433d9,Commercial,1696.329992,YH090703-05,,#FF8DA2,658


In [12]:
# Create new data frame
df = pd.DataFrame({'building': df_model.Building, 'area': df_model.Area,'facade': df_model.Facade })
df.head()

,building,area,facade
0,,1696.329992,658
1,Tower 03,2896.125039,861
2,Tower 03,2056.249041,725
3,Tower 03,2056.249041,725
4,,1696.329992,658


In [13]:
# dropping empty values for building column
df = df[df.building != ' ']

# group and sort
df = df.groupby('building').sum(False).reset_index().sort_values('building')
df

# Turn area into int
df.area = df.area.astype(int)

In [14]:
df

,building,area,facade
0,Podium,107880,48801
1,Tower 01,110141,58596
2,Tower 02,65858,24131
3,Tower 03,38228,11873


## 3. `get_building_floors` *speak with RP (need levels)* 

this method should return a dataframe that has the following features: \
`building`, `height`, `floors` 

In [15]:
# df_model instance
df_model = data['hz_model'].copy().drop(columns=['ID','Colour'])

# dropping empty values for building column
df_model = df_model[df_model.Building != ' ']

df_model.head(3)


,Typology,Area,Plot,Building
1,Commercial,2896.125039,YH090703-04,Tower 03
2,Commercial,2056.249041,YH090703-04,Tower 03
3,Commercial,2056.249041,YH090703-04,Tower 03


doesnt really work without levels because you can have multiple typologies on the same level...

In [16]:
df_building_count = df_model.groupby('Building').size().reset_index(name='floor_count')
df_building_count

,Building,floor_count
0,Podium,147
1,Tower 01,123
2,Tower 02,35
3,Tower 03,15


## 4. `get_building_areas` *speak with RP*

this method should return a dataframe that has the following features: \
`building`, `area`, `efficiency`, `gfa` (need to use coefficients)

In [17]:
# df_model instance
df_model = data['hz_model'].copy().drop(columns=['ID','Colour'])

# dropping empty values for building column
df_model = df_model[df_model.Building != ' ']

df_model.head()

,Typology,Area,Plot,Building
1,Commercial,2896.125039,YH090703-04,Tower 03
2,Commercial,2056.249041,YH090703-04,Tower 03
3,Commercial,2056.249041,YH090703-04,Tower 03
5,Commercial,2896.125039,YH090703-04,Tower 03
6,Commercial,2896.125039,YH090703-04,Tower 03


In [18]:
# rename column to match coeff for merge
df_model.rename(columns={'Typology': 'asset'}, inplace=True)
df_model

,asset,Area,Plot,Building
1,Commercial,2896.125039,YH090703-04,Tower 03
2,Commercial,2056.249041,YH090703-04,Tower 03
3,Commercial,2056.249041,YH090703-04,Tower 03
5,Commercial,2896.125039,YH090703-04,Tower 03
6,Commercial,2896.125039,YH090703-04,Tower 03
...,...,...,...,...
564,Retail,70.068490,YH090703-01,Podium
565,Retail,106.068490,YH090703-01,Podium
566,Retail,70.068583,YH090703-01,Podium
567,Retail,192.468583,YH090703-01,Podium


In [19]:
df_model.asset.unique()

array(['Commercial', 'Hotel', 'Landscape', 'Plaza', 'Retail'],
      dtype=object)

In [20]:
# Load efficiency from coeff file
df_coef = data['ksa_coefficients'].copy()
df_coef = df_coef[['asset','efficiency']]
df_coef

,asset,efficiency
0,Community,1.00
1,Hotel,0.70
2,Office,0.85
3,Residential,0.82
4,Retail,0.80
5,Sports,0.75
6,Utility,1.00
7,Transport,1.00
8,Service,0.00
9,Plaza,0.00


In [21]:
# Merge df_model with df_coef
df_merged = pd.merge(df_model,df_coef, on='asset', how='left')
df_merged = df_merged[(df_merged.asset != 'Commercial') & (df_merged.asset != 'Landscape') ]

In [22]:
# create gfa by muliplying efficiency with area
df_merged['gfa'] = (df_merged.Area * df_merged.efficiency).astype(int)
df_merged

,asset,Area,Plot,Building,efficiency,gfa
159,Hotel,802.011372,YH090703-07,Tower 01,0.7,561
230,Plaza,8741.704161,YH090703-01,Podium,0.0,0
231,Plaza,1598.578692,YH090703-04,Podium,0.0,0
232,Plaza,2878.901141,YH090703-04,Podium,0.0,0
233,Plaza,6281.033957,YH090703-04,Podium,0.0,0
...,...,...,...,...,...,...
315,Retail,70.068490,YH090703-01,Podium,0.8,56
316,Retail,106.068490,YH090703-01,Podium,0.8,84
317,Retail,70.068583,YH090703-01,Podium,0.8,56
318,Retail,192.468583,YH090703-01,Podium,0.8,153


In [23]:
# group and aggregate 
df_merged.drop(columns=['Plot'], inplace=True)
df_merged = df_merged.groupby('Building').sum(False).reset_index().sort_values('Building')
df_merged

,Building,asset,Area,efficiency,gfa
0,Podium,PlazaPlazaPlazaPlazaPlazaPlazaPlazaPlazaRetail...,84653.973603,53.6,40409
1,Tower 01,HotelRetailRetailRetailRetailRetailRetailRetai...,13180.023404,12.7,10455


In [25]:
# create clean df
df = pd.DataFrame({'building':df_merged.Building, 'area':df_merged.Area, 'efficiency':df_merged.efficiency, 'gfa':df_merged.gfa,})
df

,building,area,efficiency,gfa
0,Podium,84653.973603,53.6,40409
1,Tower 01,13180.023404,12.7,10455
